In [1]:
import kagglehub
from pathlib import Path
import shutil

c:\Users\ASUS\Desktop\Coding\CanProjects\ArtificialLearningFIFA\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
paths = [kagglehub.dataset_download("stefanoleone992/ea-sports-fc-24-complete-player-dataset"),
         kagglehub.dataset_download("nyagami/ea-sports-fc-25-database-ratings-and-stats"),
         kagglehub.dataset_download("hubertsidorowicz/football-players-stats-2024-2025")]

In [3]:
raw_dir = Path("../data/raw")
for path in paths:
    download_dir = Path(path)
    if not download_dir.exists():
        print(f"Download directory {download_dir} does not exist, skipping.")
        continue
    dataset_name = download_dir.name
    dataset_raw_dir = raw_dir / dataset_name
    dataset_raw_dir.mkdir(parents=True, exist_ok=True)
    for item in download_dir.iterdir():
        if item.is_file():
            dest = dataset_raw_dir / item.name
            shutil.copy2(item, dest)

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
from fuzzywuzzy import fuzz, process

c:\Users\ASUS\Desktop\Coding\CanProjects\ArtificialLearningFIFA\.venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
ratings_24_path = Path("../data/raw/4/male_players.csv")
ratings_25_path = Path("../data/raw/5/male_players.csv")

ratings_24_df = pd.read_csv(ratings_24_path)
ratings_25_df = pd.read_csv(ratings_25_path)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_182596\2060328440.py:4: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_24_df = pd.read_csv(ratings_24_path)


In [6]:
ratings_24_df.head()

,player_id,player_url,fifa_version,fifa_update,update_as_of,short_name,long_name,player_positions,overall,potential,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,231747,/player/231747/kylian-mbappe/240002,24.0,2.0,2023-09-22,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,94,...,63+3,63+3,63+3,68+3,63+3,54+3,54+3,54+3,63+3,18+3
1,239085,/player/239085/erling-haaland/240002,24.0,2.0,2023-09-22,E. Haaland,Erling Braut Haaland,ST,91,94,...,63+3,63+3,63+3,62+3,60+3,62+3,62+3,62+3,60+3,19+3
2,192985,/player/192985/kevin-de-bruyne/240002,24.0,2.0,2023-09-22,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,...,80+3,80+3,80+3,79+3,75+3,70+3,70+3,70+3,75+3,21+3
3,158023,/player/158023/lionel-messi/240002,24.0,2.0,2023-09-22,L. Messi,Lionel Andrés Messi Cuccittini,"CF, CAM",90,90,...,63+3,63+3,63+3,64+3,59+3,49+3,49+3,49+3,59+3,19+3
4,165153,/player/165153/karim-benzema/240002,24.0,2.0,2023-09-22,K. Benzema,Karim Benzema,"CF, ST",90,90,...,64+3,64+3,64+3,64+3,60+3,55+3,55+3,55+3,60+3,18+3


In [7]:
ratings_25_df.head()

,Unnamed: 0.1,Unnamed: 0,Rank,Name,OVR,PAC,SHO,PAS,DRI,DEF,...,Nation,League,Team,play style,url,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,0,0,1,Kylian Mbappé,91,97,90,80,92,36,...,France,LALIGA EA SPORTS,Real Madrid,"Quick Step+, Acrobatic, Finesse Shot, Flair, R...",https://www.ea.com/games/ea-sports-fc/ratings/...,NaN,NaN,NaN,NaN,NaN
1,1,1,2,Rodri,91,66,80,86,84,87,...,Spain,Premier League,Manchester City,"Tiki Taka+, Aerial, Bruiser, Long Ball Pass, P...",https://www.ea.com/games/ea-sports-fc/ratings/...,NaN,NaN,NaN,NaN,NaN
2,2,2,4,Erling Haaland,91,88,92,70,81,45,...,Norway,Premier League,Manchester City,"Acrobatic+, Bruiser, Power Header, Power Shot,...",https://www.ea.com/games/ea-sports-fc/ratings/...,NaN,NaN,NaN,NaN,NaN
3,3,3,5,Jude Bellingham,90,80,87,83,88,78,...,England,LALIGA EA SPORTS,Real Madrid,"Relentless+, Flair, Intercept, Slide Tackle, T...",https://www.ea.com/games/ea-sports-fc/ratings/...,NaN,NaN,NaN,NaN,NaN
4,4,4,7,Vini Jr.,90,95,84,81,91,29,...,Brazil,LALIGA EA SPORTS,Real Madrid,"Quick Step+, Chip Shot, Finesse Shot, First To...",https://www.ea.com/games/ea-sports-fc/ratings/...,NaN,NaN,NaN,NaN,NaN


In [8]:
def transform_name_to_short_format(name):
    if pd.isna(name):
        return name
    
    name_parts = str(name).strip().split()
    
    if len(name_parts) < 2:
        return name
    
    first_initial = name_parts[0][0].upper() + "."
    
    return f"{first_initial} {' '.join(name_parts[1:])}"

In [9]:
ratings_25_df['transformed_name'] = ratings_25_df['Name'].apply(transform_name_to_short_format)

In [13]:
names_25 = ratings_25_df['transformed_name'].dropna().unique()
names_24 = ratings_24_df['short_name'].dropna().unique()
threshold = 0.80

# 1. Find exact matches first
exact_matches = set(names_25) & set(names_24)
matches = []

for name_25 in exact_matches:
    matches.append({
        'name_25': name_25,
        'name_24': name_25,
        'similarity_score': 100,
        'original_name_25': ratings_25_df[ratings_25_df['transformed_name'] == name_25]['Name'].iloc[0] if 'Name' in ratings_25_df.columns else name_25
    })

In [18]:
list(exact_matches)[:5]

['J. Mullins', 'R. Fernandes', 'D. Crowley', 'C. Boukhalfa', 'I. Ndiaye']

In [19]:
names_25_remaining = set(names_25) - exact_matches
names_24_remaining = set(names_24) - exact_matches

In [ ]:
len(names_24_remaining)

35373

In [21]:
len(names_25_remaining)

3444

In [22]:
from tqdm import tqdm

for name_25 in tqdm(names_25_remaining, desc="Fuzzy matching"):
    best_match = process.extractOne(name_25, list(names_24_remaining), scorer=fuzz.ratio)
    if best_match and best_match[1] >= threshold * 100:
        matches.append({
            'name_25': name_25,
            'name_24': best_match[0],
            'similarity_score': best_match[1],
            'original_name_25': ratings_25_df[ratings_25_df['transformed_name'] == name_25]['Name'].iloc[0] if 'Name' in ratings_25_df.columns else name_25
        })

matches_df = pd.DataFrame(matches)
matches_df = matches_df.sort_values('similarity_score', ascending=False)

print(f"Found {len(matches_df)} matches (exact + fuzzy) above threshold {threshold}")
print(matches_df.head(20))

Fuzzy matching: 100%|██████████| 3444/3444 [28:01<00:00,  2.05it/s]

Found 14234 matches (exact + fuzzy) above threshold 0.8
              name_25         name_24  similarity_score    original_name_25
12902    J. Faye Lund    J. Faye-Lund               100    Julian Faye Lund
56        N. Gallardo     N. Gallardo               100     Nahuel Gallardo
57         M. Lemmens      M. Lemmens               100        Mats Lemmens
58          S. Afrifa       S. Afrifa               100      Stephen Afrifa
59     M. Cvjetinovic  M. Cvjetinovic               100  Mladen Cvjetinovic
60         M. Wallace      M. Wallace               100      Murray Wallace
61          M. Taylor       M. Taylor               100        Matty Taylor
62      V. Wernersson   V. Wernersson               100   Victor Wernersson
63           H. Bandé        H. Bandé               100       Hassane Bandé
48         A. Pereira      A. Pereira               100     Andreas Pereira
49          B. Pittón       B. Pittón               100        Bruno Pittón
50              Ángel           

In [ ]:
column_mapping = {
    # Player Identity
    'short_name': 'Name',
    'age': 'Age', 
    'nationality_name': 'Nation',
    'club_name': 'Team',
    'league_name': 'League',
    'player_positions': 'Position',
    'preferred_foot': 'Preferred foot',
    'weak_foot': 'Weak foot',
    'skill_moves': 'Skill moves',
    'height_cm': 'Height',
    'weight_kg': 'Weight',
    
    # Main Stats
    'overall': 'OVR',
    'pace': 'PAC',
    'shooting': 'SHO',
    'passing': 'PAS',
    'dribbling': 'DRI',
    'defending': 'DEF',
    'physic': 'PHY',
    
    # Detailed Stats
    'movement_acceleration': 'Acceleration',
    'movement_sprint_speed': 'Sprint Speed',
    'mentality_positioning': 'Positioning',
    'attacking_finishing': 'Finishing',
    'power_shot_power': 'Shot Power',
    'power_long_shots': 'Long Shots',
    'attacking_volleys': 'Volleys',
    'mentality_penalties': 'Penalties',
    'mentality_vision': 'Vision',
    'attacking_crossing': 'Crossing',
    'skill_fk_accuracy': 'Free Kick Accuracy',
    'attacking_short_passing': 'Short Passing',
    'skill_long_passing': 'Long Passing',
    'skill_curve': 'Curve',
    'skill_dribbling': 'Dribbling',
    'movement_agility': 'Agility',
    'movement_balance': 'Balance',
    'movement_reactions': 'Reactions',
    'skill_ball_control': 'Ball Control',
    'mentality_composure': 'Composure',
    'mentality_interceptions': 'Interceptions',
    'attacking_heading_accuracy': 'Heading Accuracy',
    'defending_marking_awareness': 'Def Awareness',
    'defending_standing_tackle': 'Standing Tackle',
    'defending_sliding_tackle': 'Sliding Tackle',
    'power_jumping': 'Jumping',
    'power_stamina': 'Stamina',
    'power_strength': 'Strength',
    'mentality_aggression': 'Aggression',
    
    # Goalkeeper Stats
    'goalkeeping_diving': 'GK Diving',
    'goalkeeping_handling': 'GK Handling',
    'goalkeeping_kicking': 'GK Kicking',
    'goalkeeping_positioning': 'GK Positioning',
    'goalkeeping_reflexes': 'GK Reflexes'
}

In [33]:
fifa24_dict = {}
for _, row in ratings_24_df.iterrows():
    short_name = row['short_name']
    if pd.notna(short_name):
        fifa24_dict[short_name] = row.to_dict()

fifa25_dict = {}
for _, row in ratings_25_df.iterrows():
    transformed_name = row['transformed_name']
    if pd.notna(transformed_name):
        fifa25_dict[transformed_name] = row.to_dict()

# Step 3: Build merged dataframe
merged_data = []

for _, match_row in matches_df.iterrows():
    name_24 = match_row['name_24']
    name_25 = match_row['name_25']
    
    # Get player data from both datasets
    player_24 = fifa24_dict.get(name_24, {})
    player_25 = fifa25_dict.get(name_25, {})
    
    # Create merged row
    merged_row = {
        'matched_name': name_24,
        'name_24': name_24,
        'name_25': name_25,
        'similarity_score': match_row['similarity_score'],
        'original_name_25': match_row['original_name_25']
    }
    
    # Add mapped columns - create separate _24 and _25 columns
    for col24, col25 in column_mapping.items():
        if col24 in player_24:
            merged_row[f"{col24}_24"] = player_24[col24]
        if col25 in player_25:
            merged_row[f"{col24}_25"] = player_25[col25]
            
    merged_data.append(merged_row)

merged_df = pd.DataFrame(merged_data)

In [36]:
import json

processed_dir = Path("../data/processed")
processed_dir.mkdir(parents=True, exist_ok=True)

merged_file_path = processed_dir / "merged_players_24_25_final.csv"
merged_df.to_csv(merged_file_path, index=False)

mapping_file_path = processed_dir / "column_mapping_24_25.json"
with open(mapping_file_path, 'w', encoding='utf-8') as f:
    json.dump(column_mapping, f, indent=2, ensure_ascii=False)
